# Imports and variables

In [6]:
# Importing important stuff
from First_start.imports import *
from First_start.multi_gpu import *
from First_start.init_vars import *
from First_start.start_functions import *

# Explanation

## Fish types

In [7]:
Image(url= "https://kaggle2.blob.core.windows.net/competitions/kaggle/5568/media/species-ref-key.jpg")

Fish groups:
"NoF":0
"ALB":1
"BET":2
"DOL":3
"LAG":4
"SHA":5
"YFT":6
"OTH":7

# Important Functions

Make_parallel(model, gpu_count): You have 4 gpus, to use all four, use this function with your keras model



# Code

In [ ]:
# Get x and y variables
x_train, y_train, labels_train, x_test, y_test, labels_test, image_df = get_xy(folders_path='Data/Kaggle/train/*',
                                                                               split=0.8,
                                                                               img_size=(224, 224))

# Let's check the shape of those variables
print (x_train.shape, y_train.shape, x_test.shape, y_test.shape)

((3021, 3, 224, 224), (3021, 8), (756, 3, 224, 224), (756, 8))


In [3]:
# Count of the image types
image_df.groupby('fish_type').count().sort_values('image_name')

,image_name
fish_type,
LAG,67
DOL,117
ARK,176
BET,200
HER,299
NoF,465
YFT,734
ALB,1719


# Models

## Idea1 (VGG16 + top_model):
Use VGG16 or VGG19 as base and train a model on top of those.

VGG16: https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3 

VGG19: https://gist.github.com/baraldilorenzo/8d096f48a1be4a2d660d

TopModel: The model that the groups wanted to train on top of the VGG16 or VGG19 predictor. (see below)



## Use VGG16, VGG19

In [ ]:
vgg16 = VGG16(include_top=False)
make_parallel(vgg16, gpu_count=4)

vgg19_base = VGG19(weights='imagenet')
# will not use all layers, as team did not want to use all layers, toen
vgg19 = Model(input=vgg19_base.input, output=vgg19_base.get_layer('block4_pool').output)

display('Done initializing models')

x_train_vgg16 = vgg16.predict(x_train)
x_test_vgg16 = vgg16.predict(x_test)

x_train_vgg19 = vgg19.predict(x_train)
x_test_vgg19 = vgg19.predict(x_test)

## Top Model

In [ ]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=(512, 14, 14)))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(8, activation='sigmoid'))
top_model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['binary_crossentropy'])

In [ ]:
top_model.fit(x_train_feat, y_train, validation_data=(x_test_feat, y_test))
# Final evaluation of the model
scores = top_model.evaluate(x_test_feat, y_test, verbose=0)
classes = top_model.predict_proba(x_test_feat, batch_size=32)

# get probability table
proba_df = pd.DataFrame(classes)
proba_df.columns = image_df.groupby('fish_type').count().index # workaround to get column names quickly and in right order

print(scores)
print(top_model.metrics_names)
print("Score: %.2f%%" % (scores[1]*10))
#with all images: 37%
print ' '
proba_df['image'] = labels
proba_df.to_csv('../Team 3/group3fantasticresults.csv')

In [5]:
# Fit the model
mg.make_parallel(model,8)
model.fit(x_train, y_train, validation_data=(x_test, y_test), nb_epoch=epochs, batch_size=32*8)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
classes = model.predict_proba(x_test, batch_size=32)

# get probability table
proba_df = pd.DataFrame(classes)
proba_df.columns = image_df.groupby('fish_type').count().index # workaround to get column names quickly and in right order

print("Accuracy: %.2f%%" % (scores[1]*100))
print ' '
proba_df

NameError: name 'mg' is not defined

In [ ]:
proba_df['img_names'] = labels_test.values

## Idea2: Use SVC

## Idea3: Use Data Augmentation (Katherine)

In [ ]:
model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(X_train, Y_train,
                                     batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test))